# Designing Machine Learning Workflows in Python

## 1. The Standard Workflow

In [39]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [4]:
credit = pd.read_csv('credit.csv')

In [5]:
# Inspect the first few lines of your data using head()
credit.head(3)

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,'<0',6,'critical/other existing credit',buy_radio_tv,1169,'no known savings','>=7',4,'male single',none,...,'real estate',67,none,own,2,skilled,1,yes,yes,good
1,'0<=X<200',48,'existing paid',buy_radio_tv,5951,'<100','1<=X<4',2,'female div/dep/mar',none,...,'real estate',22,none,own,1,skilled,1,none,yes,bad
2,'no checking',12,'critical/other existing credit',education,2096,'<100','4<=X<7',2,'male single',none,...,'real estate',49,none,own,1,'unskilled resident',2,none,yes,good


In [25]:
# select all columns except float based 
non_numeric_columns = [col for col in credit.select_dtypes(exclude ='int64').columns]

In [27]:
# Create a label encoder for each column. Encode the values
for column in non_numeric_columns:
    le = LabelEncoder()
    credit[column] = le.fit_transform(credit[column])

In [28]:
# Inspect the data types of the columns of the data frame
print(credit.dtypes)

checking_status           int32
duration                  int64
credit_history            int32
purpose                   int32
credit_amount             int64
savings_status            int32
employment                int32
installment_commitment    int64
personal_status           int32
other_parties             int32
residence_since           int64
property_magnitude        int32
age                       int64
other_payment_plans       int32
housing                   int32
existing_credits          int64
job                       int32
num_dependents            int64
own_telephone             int32
foreign_worker            int32
class                     int32
dtype: object


In [33]:
X, y = credit.drop(['class'], 1), credit['class']

In [34]:
# Split the data into train and test, with 20% as test
X_train, X_test, y_train, y_test = train_test_split(
  X, y, test_size=0.2, random_state=1)

In [41]:
# Create a random forest classifier, fixing the seed to 2
rf_model = RandomForestClassifier(random_state=2).fit(
  X_train, y_train)

# Use it to predict the labels of the test data
rf_predictions = rf_model.predict(X_test)

C:\Users\eturk\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [42]:
from sklearn.metrics import accuracy_score
accuracies = {}
accuracies['rf'] = accuracy_score(y_test, rf_predictions)

In [44]:
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.model_selection import GridSearchCV
#np.mean(cross_val_score(RandomForestClassifier(), X, y))

In [46]:
# Set a range for n_estimators from 10 to 40 in steps of 10
param_grid = {'n_estimators': range(10, 50, 10)}

# Optimize for a RandomForestClassifier() using GridSearchCV
grid = GridSearchCV(RandomForestClassifier(), param_grid, cv=3)
grid.fit(X, y)
grid.best_params_

{'n_estimators': 40}

In [48]:
from sklearn.ensemble import AdaBoostClassifier

# Define a grid for n_estimators ranging from 1 to 10
param_grid = {'n_estimators': range(1, 11)}

# Optimize for a AdaBoostClassifier() using GridSearchCV
grid = GridSearchCV(AdaBoostClassifier(), param_grid, cv=3)
grid.fit(X, y)
grid.best_params_

{'n_estimators': 10}

In [50]:
from sklearn.neighbors import KNeighborsClassifier

# Define a grid for n_neighbors with values 10, 50 and 100
param_grid = {'n_neighbors': [10, 50, 100]}

# Optimize for KNeighborsClassifier() using GridSearchCV
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=3)
grid.fit(X, y)
grid.best_params_

{'n_neighbors': 50}

**Categorical Encodings**

In [51]:
# Create numeric encoding for credit_history
credit_history_num = LabelEncoder().fit_transform(
  credit['credit_history'])

# Create a new feature matrix including the numeric encoding
X_num = pd.concat([X, pd.Series(credit_history_num)], 1)

# Create new feature matrix with dummies for credit_history
X_hot = pd.concat(
  [X, pd.get_dummies(credit['credit_history'])], 1)

# Compare the number of features of the resulting DataFrames
print(X_hot.shape[1] > X_num.shape[1])

True


**Feature Transformations**

You are discussing the credit dataset with the bank manager. She suggests that the safest loan applications tend to request mid-range credit amounts. Values that are either too low or too high suggest high risk. This means that a non-linear relationship might exist between this variable and the class. You want to test this hypothesis. You will construct a non-linear transformation of the feature. Then, you will assess which of the two features is better at predicting the class using SelectKBest() and the chi2() metric.

In [52]:
from sklearn.feature_selection import SelectKBest, chi2

In [53]:
# Function computing absolute difference from column mean
def abs_diff(x):
    return np.abs(x-np.mean(x))

# Apply it to the credit amount and store to new column
credit['diff'] = abs_diff(credit['credit_amount'])

# Create a feature selector with chi2 that picks one feature
sk = SelectKBest(chi2, k=1)

# Use the selector to pick between credit_amount and diff
sk.fit(credit[['credit_amount', 'diff']], credit['class'])

# Inspect the results
sk.get_support()

array([ True, False])

**Bringing it all together**

In [54]:
arr = pd.read_csv('arrh.csv')

In [55]:
arr.head()

,age,sex,height,weight,QRSduration,PRinterval,Q-Tinterval,Tinterval,Pinterval,QRS,...,chV6_QwaveAmp,chV6_RwaveAmp,chV6_SwaveAmp,chV6_RPwaveAmp,chV6_SPwaveAmp,chV6_PwaveAmp,chV6_TwaveAmp,chV6_QRSA,chV6_QRSTA,class
0,75,0,190,80,91,193,371,174,121,-16,...,0.0,9.0,-0.9,0.0,0.0,0.9,2.9,23.3,49.4,0
1,56,1,165,64,81,174,401,149,39,25,...,0.0,8.5,0.0,0.0,0.0,0.2,2.1,20.4,38.8,0
2,54,0,172,95,138,163,386,185,102,96,...,0.0,9.5,-2.4,0.0,0.0,0.3,3.4,12.3,49.0,0
3,55,0,175,94,100,202,380,179,143,28,...,0.0,12.2,-2.2,0.0,0.0,0.4,2.6,34.6,61.6,1
4,75,0,190,80,88,181,360,177,103,-16,...,0.0,13.1,-3.6,0.0,0.0,-0.1,3.9,25.4,62.8,0


In [56]:
arr.shape

(452, 280)

In [66]:
from sklearn.ensemble import RandomForestClassifier 

# Find from sklearn.ensemble import RandomForestClassifierhe best value for max_depth among values 2, 5 and 10
grid_search = GridSearchCV(
  RandomForestClassifier(random_state=1), param_grid={'max_depth': [2, 5, 10]})

best_value = grid_search.fit(
  X_train, y_train).best_params_['max_depth']

# Using the best value from above, fit a random forest
clf = RandomForestClassifier(
  random_state=1, max_depth=best_value).fit(X_train, y_train)

# Apply SelectKBest with chi2 and pick top 100 features
vt = SelectKBest(chi2, k=20).fit(X_train, y_train)

# Create a new dataset only containing the selected features
X_train_reduced = vt.transform(X_train)

C:\Users\eturk\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\eturk\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\eturk\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\eturk\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\eturk\Anaconda3\lib\s

In [65]:
X_train_reduced.shape

(800, 20)

## The Human in the Loop

In [87]:
flows = pd.read_csv('lanl_flows.csv')

In [89]:
flows.columns.values

array(['time', 'duration', 'source_computer', 'source_port',
       'destination_computer', 'destination_port', 'protocol',
       'packet_count', 'byte_count'], dtype=object)

In [85]:
# Unit of analysis = destination_computer

flows_grouped = flows.groupby('destination_computer')

# Access one of its elements you have to call list()
list(flows_grouped)[0]

# Convert each of these DataFrames to feature vectors.
def featurize(df):
    return {
        'unique_ports': len(set(df['destination_port'])),
        'average_packet': np.mean(df['packet_count']),
        'average_duration': np.mean(df['duration'])
    }

# Applying this function on each group yields an iterator containing one vector per group, all of the same length. 
out = flows.groupby('destination_computer').apply(featurize)

# Create DF one row for destination computer
X = pd.DataFrame(list(out), index=out.index)
X.head()

bads = set(attacks['source_computer'].append(attacks['destination_computer']))
y = [x in bads for x in X.index]

X_train, X_test, y_train, y_test = train_test_split(X, y)
clf = AdaBoostClassifier()
accuracy_score(y_test, clf.fit(X_train, y_train).predict(X_test))

**Is the source or the destination bad?**

In the previous lesson, you used the destination computer as your entity of interest. However, your cybersecurity analyst just told you that it is the infected machines that generate the bad traffic, and will therefore appear as a source, not a destination, in the flows dataset.

In [ ]:
# Convert each of these DataFrames to feature vectors.
def featurize(df):
    return {
        'unique_ports': len(set(df['destination_port'])),
        'average_packet': np.mean(df['packet_count']),
        'average_duration': np.mean(df['duration'])
    }

# Group by source computer, and apply the feature extractor 
out = flows.groupby('source_computer').apply(featurize)

# Convert the iterator to a dataframe by calling list on it
X = pd.DataFrame(list(out), index=out.index)

bads = set(attacks['source_computer'].append(attacks['destination_computer']))
y = [x in bads for x in X.index]

# Check which sources in X.index are bad to create labels
y = [x in bads for x in X.index]

# Report the average accuracy of Adaboost over 3-fold CV
print(np.mean(cross_val_score(AdaBoostClassifier(), X, y)))

In [ ]:
# Create a feature counting unique protocols per source
protocols = flows.groupby('source_computer').apply(
  lambda df: len(set(df['protocol'])))

# Convert this feature into a dataframe, naming the column
protocols_DF = pd.DataFrame(
  protocols, index=protocols.index, columns=['protocol'])

# Now concatenate this feature with the previous dataset, X
X_more = pd.concat([X, protocols_DF], axis=1)

# Refit the classifier and report its accuracy
print(np.mean(cross_val_score(
  AdaBoostClassifier(), X_more, y)))

**imperfect labels**

In [ ]:
# From features to labels

# Convert a feature into a labeling heuristic:

X_train, X_test, y_train, y_test = train_test_split(X, y)
y_weak_train = X_train['unique_ports'] > 15

X_train_aug = pd.concat([X_train, X_train])
y_train_aug = pd.concat([pd.Series(y_train), pd.Series(y_weak_train)])

weights = [1.0]*len(y_train) + [0.1]*len(y_weak_train)

Turning a heuristic into a classifier
 
You are surprised by the fact that heuristics can be so helpful. So you decide to treat the heuristic that "too many unique ports is suspicious" as a classifier in its own right. You achieve that by thresholding the number of unique ports per source by the average number used in bad source computers -- these are computers for which the label is True. 

In [ ]:
# Create a new dataset X_train_bad by subselecting bad hosts
X_train_bad = X_train[y_train]

# Calculate the average of unique_ports in bad examples
avg_bad_ports = np.mean(X_train_bad['unique_ports'])

# Label as positive sources that use more ports than that
pred_port = X_test['unique_ports'] > avg_bad_ports

# Print the accuracy of the heuristic
print(accuracy_score(y_test, pred_port))

Combining heuristics

A different cyber analyst tells you that during certain types of attack, the infected source computer sends small bits of traffic, to avoid detection. This makes you wonder whether it would be better to create a combined heuristic that simultaneously looks for large numbers of ports and small packet sizes. Does this improve performance over the simple port heuristic?

In [ ]:
# Compute the mean of average_packet for bad sources
avg_bad_packet = np.mean(X_train[y_train]['average_packet'])

# Label as positive if average_packet is lower than that
pred_packet = X_test['average_packet'] < avg_bad_packet

# Find indices where pred_port and pred_packet both True
pred_port = X_test['unique_ports'] > avg_bad_ports
pred_both = pred_packet * pred_port

# Ports only produced an accuracy of 0.919. Is this better?
print(accuracy_score(y_test, pred_both))

Dealing with label noise

One of your cyber analysts informs you that many of the labels for the first 100 source computers in your training data might be wrong because of a database error. She hopes you can still use the data because most of the labels are still correct, but asks you to treat these 100 labels as "noisy". Thankfully you know how to do that, using weighted learning.

In [ ]:
# Fit a Gaussian Naive Bayes classifier to the training data
clf = GaussianNB().fit(X_train, y_train_noisy)

# Report its accuracy on the test data
print(accuracy_score(y_test, clf.predict(X_test)))

# Assign half the weight to the first 100 noisy examples
weights = [0.5]*100 + [1.0]*(len(y_train_noisy)-100)

# Refit using weights and report accuracy. Has it improved?
clf_weights = GaussianNB().fit(X_train, y_train_noisy, sample_weight=weights)
print(accuracy_score(y_test, clf_weights.predict(X_test)))

**Loss Functions Part I**

In [ ]:
# Binarize label
kdd['label'] = kdd['label'] == 'bad'

# Fit a Gaussian Naive Bayes classier:
clf = GaussianNB().fit(X_train, y_train)
predictions = clf.predict(X_test)
results = pd.DataFrame({
    'actual': y_test,
    'predicted': predictions
})

# Confusion Matrix
conf_mat = confusion_matrix(ground_truth, predictions)
tn, fp, fn, tp = conf_mat.ravel()

# Scalar performance metrics
accuracy = 1-(fp + fn)/len(ground_truth)  # (tp + tn)/len(y_test)
recall = tp/(tp+fn)
fpr = fp/(tn+fp)
precision = tp/(tp+fp)
f1 = 2*(precision*recall)/(precision+recall)

accuracy_score(ground_truth, predictions)
recall_score(ground_truth, predictions)
precision_score(ground_truth, predictions)
f1_score(ground_truth, predictions)

Real-world cost analysis

You will still work on the credit dataset for this exercise. Recall that a "positive" in this dataset means "bad credit", i.e., a customer who defaulted on their loan, and a "negative" means a customer who continued to pay without problems. The bank manager informed you that the bank makes 10K profit on average from each "good risk" customer, but loses 150K from each "bad risk" customer. Your algorithm will be used to screen applicants, so those that are labeled as "negative" will be given a loan, and the "positive" ones will be turned down. What is the total cost of your classifier? 

In [ ]:
# Fit a random forest classifier to the training data
clf = RandomForestClassifier(random_state=2).fit(X_train, y_train)

# Label the test data
preds = clf.predict(X_test)

# Get false positives/negatives from the confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()

# Now compute the cost using the manager's advice
cost = fp*10 + fn*150

**Loss Function II**

In [ ]:
clf = GaussianNB().fit(X_train, y_train)
scores = clf.predict_proba(X_test)

[s[1] > 0.5 for s in scores] == clf.predict(X_test)

# ROC Curves
fpr, tpr, thres = roc_curve(
ground_truth,
[s[1] for s in scores])
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

# AUC
clf = AdaBoostClassifier().fit(X_train, y_train)
scores_ab = clf.predict_proba(X_test)
roc_auc_score(ground_truth, [s[1] for s in scores_ab])

# Cost Minimisation
def my_scorer(y_test, y_est, cost_fp=10.0, cost_fn=1.0):
tn, fp, fn, tp = confusion_matrix(y_test, y_est).ravel()
return cost_fp*fp + cost_fn*fn

t_range = [0.0, 0.25, 0.5, 0.75, 1.0]
costs = [
my_scorer(y_test, [s[1] > thres for s in scores]) for thres in t_range
]

Default thresholding

You would like to confirm that the DecisionTreeClassifier() uses the same default classification threshold as mentioned in the previous lesson, namely 0.5. It seems strange to you that all classifiers should use the same threshold.

In [ ]:
# Score the test data using the given classifier
scores = clf.predict_proba(X_test)

# Get labels from the scores using the default threshold
preds = [s[1] > 0.5 for s in scores]

# Use the predict method to label the test data again
preds_default = clf.predict(X_test)

# Compare the two sets of predictions
all(preds == preds_default)

Optimizing the threshold

You heard that the default value of 0.5 maximizes accuracy in theory, but you want to test what happens in practice. So you try out a number of different threshold values, to see what accuracy you get, and hence determine the best-performing threshold value. You repeat this experiment for the F1 score. Is 0.5 the optimal threshold? Is the optimal threshold for accuracy and for the F1 score the same? Go ahead and find out! You have a scores matrix available, obtained by scoring the test data. 

In [ ]:
# Create a range of equally spaced threshold values
t_range = [0.0, 0.25, 0.5, 0.75, 1.0]

# Store the predicted labels for each value of the threshold
preds = [[s[1] > thr for s in scores] for thr in t_range]

# Compute the accuracy for each threshold
accuracies = [accuracy_score(y_test, p) for p in preds]

# Compute the F1 score for each threshold
f1_scores = [f1_score(y_test, p) for p in preds]

# Report the optimal threshold for accuracy, and for F1
print(t_range[argmax(accuracies)], t_range[argmax(f1_scores)])

Bringing it all together

One of the engineers in your arrhythmia detection startup rushes into your office to let you know that there is a problem with the ECG sensor for overweight users. You decide to reduce the influence of examples with weight over 80 by 50%. You are also told that since your startup is targeting the fitness market and makes no medical claims, scaring an athlete unnecessarily is costlier than missing a possible case of arrhythmia. You decide to create a custom loss that makes each "false alarm" ten times costlier than missing a case of arrhythmia. Does down-weighting overweight subjects improve this custom loss?

In [ ]:
# Create a scorer assigning more cost to false positives
def my_scorer(y_test, y_est, cost_fp=10.0, cost_fn=1.0):
    tn, fp, fn, tp = confusion_matrix(y_test, y_est).ravel()
    return cost_fp*fp + cost_fn*fn

# Fit a DecisionTreeClassifier to the data and compute the loss
clf = DecisionTreeClassifier(random_state=2).fit(X_train, y_train)
print(my_scorer(y_test, clf.predict(X_test)))

# Refit, downweighting subjects whose weight is above 80
weights = [0.5 if w > 80 else 1.0 for w in X_train.weight]
clf_weighted = DecisionTreeClassifier(random_state=2).fit(
  X_train, y_train, sample_weight=weights)
print(my_scorer(y_test, clf_weighted.predict(X_test)))

## Model Lifecycle Management

**From Workflows to Pipeline**

In [ ]:
from sklearn.ensemble import RandomForestClassifier as rf
X_train, X_test, y_train, y_test = train_test_split(X, y)

grid_search = GridSearchCV(rf(), param_grid={'max_depth': [2, 5, 10]})
grid_search.fit(X_train, y_train)
depth = grid_search.best_params_['max_depth']

vt = SelectKBest(f_classif, k=3).fit(X_train, y_train)
clf = rf(max_depth=best_value).fit(vt.transform(X_train), y_train)
accuracy_score(clf.predict(vt.transform(X_test), y_test))

# Optimize max_depth
pg = {'max_depth': [2,5,10]}
gs = GridSearchCV(rf(),
param_grid=pg)
gs.fit(X_train, y_train)
depth = gs.best_params_['max_depth']

# Optimize n_estimators
pg = {'n_estimators': [10,20,30]}
gs = GridSearchCV(
rf(max_depth=depth),
param_grid=pg)
gs.fit(X_train, y_train)
n_est = gs.best_params_[
'n_estimators']

# Jointly max_depth and n_estimators
pg = {
'max_depth': [2,5,10],
'n_estimators': [10,20,30]
}
gs = GridSearchCV(rf(),
param_grid=pg)
gs.fit(X_train, y_train)
print(gs.best_params_)

# Pipelines
from sklearn.pipeline import Pipeline
pipe = Pipeline([
('feature_selection'
, SelectKBest(f_classif)),
('classifier'
, RandomForestClassifier())
])

params = dict(
feature_selection__k=[2, 3, 4],
classifier__max_depth=[5, 10, 20]
)

grid_search = GridSearchCV(pipe, param_grid=params)
gs = grid_search.fit(X_train, y_train).best_params_

# Customizing your pipeline
from sklearn.metrics import roc_auc_score, make_scorer
auc_scorer = make_scorer(roc_auc_score)
grid_search = GridSearchCV(pipe, param_grid=params, scoring=auc_scorer)

Your first pipeline 

Back in the arrhythmia startup, your monthly review is coming up, and as part of that an expert Python programmer will be reviewing your code. You decide to tidy up by following best practices and replace your script for feature selection and random forest classification, with a pipeline. 

In [ ]:
# Create pipeline with feature selector and classifier
pipe = Pipeline([
    ('feature_selection', SelectKBest(f_classif)),
    ('clf', RandomForestClassifier(random_state=2))])

# Create a parameter grid
params = {
   'feature_selection__k':[10, 20],
   'clf__n_estimators':[2, 5]}

# Initialise the grid search object
grid_search = GridSearchCV(pipe, param_grid=params)

# Fit it to the data and print the best value combination
print(grid_search.fit(X_train, y_train).best_params_)

Custom scorers in pipelines

You are proud of the improvement in your code quality, but just remembered that previously you had to use a custom scoring metric in order to account for the fact that false positives are costlier to your startup than false negatives. You hence want to equip your pipeline with scorers other than accuracy, including roc_auc_score(), f1_score(), and you own custom scoring function. 

In [ ]:
# Create a custom scorer
scorer = make_scorer(roc_auc_score)

# Initialize the CV object
gs = GridSearchCV(pipe, param_grid=params, scoring=scorer)

# Fit it to the data and print the winning combination
print(gs.fit(X_train, y_train).best_params_)

In [ ]:
# Create a custom scorer
scorer = make_scorer(f1_score)

# Initialise the CV object
gs = GridSearchCV(pipe, param_grid=params, scoring=scorer)

# Fit it to the data and print the winning combination
print(gs.fit(X_train, y_train).best_params_)

In [ ]:
# Create a custom scorer
scorer = make_scorer(my_metric)

# Initialise the CV object
gs = GridSearchCV(pipe, param_grid=params, scoring=scorer)

# Fit it to the data and print the winning combination
print(gs.fit(X_train, y_train).best_params_)

**Model Deployment**

In [ ]:
# Serializing the model: 

#Store a classier to file:
import pickle
clf = RandomForestClassifier().fit(X_train, y_train)

with open('model.pkl', 'wb') as file:
    pickle.dump(clf, file=file)

# Load it again from file:
with open('model.pkl', 'rb') as file:
    clf2 = pickle.load(file)

In [ ]:
# Serializing your pipeline:

# Development environment

vt = SelectKBest(f_classif).fit(X_train, y_train)
clf = RandomForestClassifier().fit(vt.transform(X_train), y_train)

with open('vt.pkl', 'wb') as file:
    pickle.dump(vt)

with open('clf.pkl','wb') as file:
    pickle.dump(clf)

In [ ]:
# Serializing your pipeline:

# Production environment

with open('vt.pkl', 'rb') as file:
    vt = pickle.load(vt)

with open('clf.pkl', 'rb') as file:
    clf = pickle.load(clf)

clf.predict(vt.transform(X_new))

In [ ]:
# Serializing your pipeline:

# Development environment

pipe = Pipeline([('fs', SelectKBest(f_classif)), ('clf', RandomForestClassifier())])
params = dict(fs__k=[2, 3, 4],
clf__max_depth=[5, 10, 20])
gs = GridSearchCV(pipe, params)
gs = gs.fit(X_train, y_train)

with open('pipe.pkl', 'wb') as file:
    pickle.dump(gs, file)

In [ ]:
# Serializing your pipeline:

# Production environment

with open('pipe.pkl', 'rb') as file:
    gs = pickle.dump(gs, file)
gs.predict(X_test)

In [ ]:
# Custom feature transformations
def negate_second_column(X):
    Z = X.copy()
    Z[:,1] = -Z[:,1]
    return Z

pipe = Pipeline([('ft', FunctionTransformer(negate_second_column)), ('clf', RandomForestClassifier())])

Pickles

Finally, it is time for you to push your first model to production. It is a random forest classifier which you will use as a baseline, while you are still working to develop a better alternative.

In [ ]:
# Fit a random forest to the training set
clf = RandomForestClassifier(random_state=42).fit(
  X_train, y_train)

# Save it to a file, to be pushed to production
with open('model.pkl', 'wb') as file:
    pickle.dump(clf, file=file)

# Now load the model from file in the production environment
with open('model.pkl', 'rb') as file:
    clf_from_file = pickle.load(file)

# Predict the labels of the test dataset
preds = clf_from_file.predict(X_test)

Custom function transformers in pipelines

At some point, you were told that the sensors might be performing poorly for obese individuals. Previously you had dealt with that using weights, but now you are thinking that this information might also be useful for feature engineering, so you decide to replace the recorded weight of an individual with an indicator of whether they are obese. You want to do this using pipelines. 

In [ ]:
# Define a feature extractor to flag very large values
def more_than_average(X, multiplier=1.0):
  Z = X.copy()
  Z[:,1] = Z[:,1] > multiplier*np.mean(Z[:,1])
  return Z

# Convert your function so that it can be used in a pipeline
pipe = Pipeline([
  ('ft', FunctionTransformer(more_than_average)),
  ('clf', RandomForestClassifier(random_state=2))])

# Optimize the parameter multiplier using GridSearchCV
params = {'ft__multiplier':[1, 2, 3]}
grid_search = GridSearchCV(pipe, param_grid=params)

**Iterating without Overfitting**

In [ ]:
# Cross-validation results

grid_search = GridSearchCV(pipe, params, cv=3, return_train_score=True)
gs = grid_search.fit(X_train, y_train)
results = pd.DataFrame(gs.cv_results_)

results[['mean_train_score','std_train_score','mean_test_score','std_test_score']]

Challenge the champion

Having pushed your random forest to production, you suddenly worry that a naive Bayes classifier might be better. You want to run a champion-challenger test, by comparing a naive Bayes, acting as the challenger, to exactly the model which is currently in production, which you will load from file to make sure there is no confusion. You will use the F1 score for assessment. 

In [ ]:
# Load the current model from disk
champion = pickle.load(open('model.pkl', 'rb'))

# Fit a Gaussian Naive Bayes to the training data
challenger = GaussianNB().fit(X_train, y_train)

# Print the F1 test scores of both champion and challenger
print(f1_score(y_test, champion.predict(X_test)))
print(f1_score(y_test, challenger.predict(X_test)))

# Write back to disk the best-performing model
with open('model.pkl', 'wb') as file:
    pickle.dump(champion, file=file)

Cross-validation statistics

You used grid search CV to tune your random forest classifier, and now want to inspect the cross-validation results to ensure you did not overfit. In particular you would like to take the difference of the mean test score for each fold from the mean training score.

In [ ]:
# Fit your pipeline using GridSearchCV with three folds
grid_search = GridSearchCV(pipe, params, cv=3, return_train_score=True)

# Fit the grid search
gs = grid_search.fit(X_train, y_train)

# Store the results of CV into a pandas dataframe
results = pd.DataFrame(gs.cv_results_)

# Print the difference between mean test and training scores
print(results['mean_test_score']-results['mean_train_score'])

**Dataset Shift**

In [ ]:
# Windows

# Sliding Window
window = (t_now-window_size+1):t_now
sliding_window = elec.loc[window]

# Expanding Window
window = 0:t_now
expanding_window = elec.loc[window]

In [ ]:
# Dataset shift detection

# t_now = 40000, window_size = 20000
clf_full = RandomForestClassifier().fit(X, y)
clf_sliding = RandomForestClassifier().fit(sliding_X, sliding_y)

# Use future data as test
test = elec.loc[t_now:elec.shape[0]]
test_X = test.drop('class', 1); test_y = test['class']

roc_auc_score(test_y, clf_full.predict(test_X))
roc_auc_score(test_y, clf_sliding.predict(test_X))

In [ ]:
# Window size
for w_size in range(10, 100, 10):
    sliding = arrh.loc[
    (t_now - w_size + 1):t_now]
    X = sliding.drop('class', 1)
    y = sliding['class']
    clf = GaussianNB()
    clf.fit(X, y)
    preds = clf.predict(test_X)
    roc_auc_score(test_y, preds)

Tuning the window size

You want to check for yourself that the optimal window size for the arrhythmia dataset is 50. You have been given the dataset as a pandas data frame called arrh, and want to use a subset of the data up to time t_now.  the possibility of dataset shift introduces yet another parameter to optimize: the window size. This cannot be done with Cross-Validation on historical data, but instead requires the technique shown here.

In [ ]:
# Loop over window sizes
for w_size in wrange:

    # Define sliding window
    sliding = arrh.loc[(t_now - w_size + 1):t_now]

    # Extract X and y from the sliding window
    X, y = sliding.drop('class', 1), sliding['class']
    
    # Fit the classifier and store the F1 score
    preds = GaussianNB().fit(X, y).predict(X_test)
    accuracies.append(f1_score(y_test, preds))

# Estimate the best performing window size
optimal_window = wrange[np.argmax(accuracies)]

Bringing it all together

You have two concerns about your pipeline at the arrhythmia detection startup:

The app was trained on patients of all ages, but is primarily being used by fitness users who tend to be young. You suspect this might be a case of domain shift, and hence want to disregard all examples above 50 years old.
You are still concerned about overfitting, so you want to see if making the random forest classifier less complex and selecting some features might help with that.

In [ ]:
# Create a pipeline 
pipe = Pipeline([
  ('ft', SelectKBest()), ('clf', RandomForestClassifier(random_state=2))])

# Create a parameter grid
grid = {'ft__k':[5, 10], 'clf__max_depth':[10, 20]}

# Execute grid search CV on a dataset containing under 50s
grid_search = GridSearchCV(pipe, param_grid=grid)
arrh = arrh.iloc[np.where(arrh['age'] < 50)]
grid_search.fit(arrh.drop('class', 1), arrh['class'])

# Push the fitted pipeline to production
with open('pipe.pkl', 'wb') as file:
    pickle.dump(grid_search, file)

## Unsupervised Workflows

**Anomaly Detection**

In [ ]:
# Local outlier factor (LoF)
from sklearn.neighbors import LocalOutlierFactor as lof
clf = lof()
y_pred = clf.fit_predict(X)

clf.negative_outlier_factor_[:4]

confusion_matrix(y_pred, ground_truth)

In [ ]:
# Local outlier factor (LoF)
clf = lof(contamination=0.02) # default is 0.1
y_pred = clf.fit_predict(X)

confusion_matrix(y_pred, ground_truth)

A simple outlier

In [97]:
# Import the LocalOutlierFactor module
from sklearn.neighbors import LocalOutlierFactor as lof

# Create the list [1.0, 1.0, ..., 1.0, 10.0] as explained
x = [1.0]*30
x.append(10.0)

# Cast to a data frame
X = pd.DataFrame(x)

# Fit the local outlier factor and print the outlier scores
print(lof().fit_predict(X)) # 1 for normal, -1 for outliers

C:\Users\eturk\Anaconda3\lib\site-packages\sklearn\neighbors\lof.py:236: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)


[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1 -1]


LoF contamination

Your medical advisor at the arrhythmia startup informs you that your training data might not contain all possible types of arrhythmia. How on earth will you detect these other types without any labeled examples? Could an anomaly detector tell the difference between healthy and unhealthy without access to labels? But first, you experiment with the contamination parameter to see its effect on the confusion matrix. You have LocalOutlierFactor as lof, numpy as np, the labels as ground_truth encoded in -1and 1 just like local outlier factor output, and the unlabeled training data as X.

In [ ]:
# Contamination to match outlier frequency in ground_truth
preds = lof(
  contamination=np.mean(ground_truth==-1.0)).fit_predict(X)

# Print the confusion matrix
print(confusion_matrix(ground_truth, preds))

**Novelty Detection**

In [ ]:
# Novelty LoF

# Workaround
preds = lof().fit_predict(np.concatenate([X_train, X_test]))
preds = preds[X_train.shape[0]:]

# Novelty LoF
clf = lof(novelty=True)
clf.fit(X_train)
y_pred = clf.predict(X_test)

clf = LocalOutlierFactor(novelty=True)
clf.fit(X_train)
y_scores = clf.score_samples(X_test)

# One-class Suport Vector Machine
clf = OneClassSVM()
clf.fit(X_train)
y_pred = clf.predict(X_test)

clf = OneClassSVM()
clf.fit(X_train)
y_scores = clf.score_samples(X_test)
threshold = np.quantile(y_scores, 0.1)
y_pred = y_scores <= threshold

# Isolation Forests
clf = IsolationForest()
clf.fit(X_train)
y_scores = clf.score_samples(X_test)

clf = LocalOutlierFactor(novelty=True)
clf.fit(X_train)
y_scores = clf.score_samples(X_test)

In [ ]:
# Comparing Detectors' Performance ---- > AUC is used 

clf_lof = LocalOutlierFactor(novelty=True).fit(X_train)
clf_isf = IsolationForest().fit(X_train)
clf_svm = OneClassSVM().fit(X_train)

roc_auc_score(y_test, clf_lof.score_samples(X_test)
              
roc_auc_score(y_test, clf_isf.score_samples(X_test))
              
roc_auc_score(y_test, clf_svm.score_samples(X_test))

A simple novelty

You find novelty detection more useful than outlier detection, but want to make sure it works on the simple example you came up with before. This time you will use a sequence of thirty examples all with value 1.0 as a training set, and try to see if the example 10.0 is labeled as a novelty. You have access to pandas as pd, and the LocalOutlierFactor module as lof.

In [98]:
# Create a list of thirty 1s and cast to a dataframe
X = pd.DataFrame([1.0]*30)

# Create an instance of a lof novelty detector
detector = lof(novelty=True)

# Fit the detector to the data
detector.fit(X)

# Use it to predict the label of an example with value 10.0
print(detector.predict(pd.DataFrame([10.0])))

C:\Users\eturk\Anaconda3\lib\site-packages\sklearn\neighbors\lof.py:236: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)


[-1]


Three novelty detectors

Finally, you know enough to run some tests on the use of a few anomaly detectors on the arrhythmia dataset. To test their performance, you will train them on an unlabeled training dataset, but then compare their predictions to the ground truth on the test data using their method .score_samples(). This time, you will be asked to import the detectors as part of the exercise, but you do have the data X_train, X_test, y_train, y_test preloaded as usual.

In [ ]:
# Import the novelty detector
from sklearn.svm import OneClassSVM as onesvm

# Fit it to the training data and score the test data
svm_detector = onesvm().fit(X_train)
scores = svm_detector.score_samples(X_test)

# Import the novelty detector
from sklearn.ensemble import IsolationForest as isof

# Fit it to the training data and score the test data
isof_detector = isof().fit(X_train)
scores = isof_detector.score_samples(X_test)

# Import the novelty detector
from sklearn.neighbors import LocalOutlierFactor as lof

# Fit it to the training data and score the test data
lof_detector = lof(novelty=True).fit(X_train)
scores = lof_detector.score_samples(X_test)

Contamination revisited

You notice that one-class SVM does not have a contamination parameter. But you know well by now that you really need a way to control the proportion of examples that are labeled as novelties in order to control your false positive rate. So you decide to experiment with thresholding the scores. 

In [ ]:
# Fit a one-class SVM detector and score the test data
nov_det = onesvm().fit(X_train)
scores = nov_det.score_samples(X_test)

# Find the observed proportion of outliers in the test data
prop = np.mean(y_test==1.0)

# Compute the appropriate threshold
threshold = np.quantile(scores, prop)

# Print the confusion matrix for the thresholded scores
print(confusion_matrix(y_test, scores > threshold))

**Distance-based learning**

In [99]:
# Distance and similarity
from sklearn.neighbors import DistanceMetric as dm
dist = dm.get_metric('euclidean')
X = [[0,1], [2,3], [0,6]]
dist.pairwise(X)

array([[0.        , 2.82842712, 5.        ],
       [2.82842712, 0.        , 3.60555128],
       [5.        , 3.60555128, 0.        ]])

In [100]:
X = np.matrix(X)
np.sqrt(np.sum(np.square(X[0,:] - X[1,:])))

2.8284271247461903

In [ ]:
# Non-Euclidean Local Outlier Factor
clf = LocalOutlierFactor(novelty=True, metric='chebyshev')
clf.fit(X_train)
y_pred = clf.predict(X_test)

dist = dm.get_metric('chebyshev')
X = [[0,1], [2,3], [0,6]]
dist.pairwise(X)

# Hamming Distance Matrix:
dist = dm.get_metric('hamming')
X = [[0,1], [2,3], [0,6]]
dist.pairwise(X)

# pdist
from scipy.spatial.distance import pdist
X = [[0,1], [2,3], [0,6]]
pdist(X, 'cityblock')

from scipy.spatial.distance import squareform
squareform(pdist(X, 'cityblock'))

Find the neighbor

It is clear that the local outlier factor algorithm depends a lot on the idea of a nearest neighbor, which in turn depends on the choice of distance metric. So you decide to experiment some more with the hepatitis dataset introduced in the previous lesson. You are given three examples stored in features, whose classes are stored in labels. You will identify the nearest neighbor to the first example (row with index 0) using three different distance metrics, Euclidean, Hamming and Chebyshev, and on the basis of that choose which distance metric to use. You will import the necessary module as part of the exercise, but pandas and numpy already available, as are features and their labels labels.

In [ ]:
# Import DistanceMetric as dm
from sklearn.neighbors import DistanceMetric as dm

# Find the Euclidean distance between all pairs
dist_eucl = dm.get_metric('euclidean').pairwise(features)

# Find the Hamming distance between all pairs
dist_hamm = dm.get_metric('hamming').pairwise(features)

# Find the Chebyshev distance between all pairs
dist_cheb = dm.get_metric('chebyshev').pairwise(features)

Not all metrics agree

In the previous exercise you saw that not all metrics agree when it comes to identifying nearest neighbors. But does this mean they might disagree on outliers, too? You decide to put this to the test. You use the same data as before, but this time feed it into a local outlier factor outlier detector. 

In [ ]:
# Compute outliers according to the Euclidean metric
out_eucl = lof(metric='euclidean').fit_predict(features)

# Compute outliers according to the Hamming metric
out_hamm = lof(metric='hamming').fit_predict(features)

# Compute outliers according to the Jaccard metric
out_jacc = lof(metric='jaccard').fit_predict(features)

# Find if all three metrics agree on any one datapoint
print(any(out_jacc + out_hamm + out_eucl == -3))

Note: There is no datapoint that all three metrics flag as an outlier. So choosing a distance metric should be done with great caution! You now have a concrete understanding of the effect of distance metrics on outlier detection.

**Unstructured Data**

In [ ]:
import stringdist
stringdist.levenshtein('abc', 'acc')

In [ ]:
stringdist.levenshtein('acc', 'cce')

In [ ]:
stringdist.levenshtein('ILSALVGIV', 'ILSALVGIL')

In [ ]:
sequences = np.array(proteins['sequence'].iloc[:3]).reshape(-1,1)

def my_levenshtein(x, y):
    return stringdist.levenshtein(x[0], y[0])

pdist(sequences, metric=my_levenshtein)

In [ ]:
# Protein outliers with precomputed matrices

# This takes 2 minutes for about 1000 examples
M = pdist(sequences, my_levenshtein)

# LoF detector with a precomputed distance matrix:
# This takes 3 seconds
detector = lof(metric='precomputed', contamination=0.1)
preds = detector.fit_predict(M)

roc_auc_score(proteins['label'] == 'VIRUS', preds == -1)

Restricted Levenshtein

You notice that the stringdist package also implements a variation of Levenshtein distance called the Restricted Damerau-Levenshtein distance, and want to try it out. You will follow the logic from the lesson, wrapping it inside a custom function and precomputing the distance matrix before fitting a local outlier factor anomaly detector.

In [ ]:
# Wrap the RD-Levenshtein metric in a custom function
def my_rdlevenshtein(u, v):
    return stringdist.rdlevenshtein(u[0], v[0])

# Reshape the array into a numpy matrix
sequences = np.array(proteins['seq']).reshape(-1, 1)

# Compute the pairwise distance matrix in square form
M = squareform(pdist(sequences, my_rdlevenshtein))

# Run a LoF algorithm on the precomputed distance matrix
preds = lof(metric='precomputed').fit_predict(M)

# Compute the accuracy of the outlier predictions
print(accuracy(proteins['label'] == 'VIRUS', preds == -1))

Bringing it all together

In addition to the distance-based learning anomaly detection pipeline you created in the last exercise, you want to also support a feature-based learning one with one-class SVM. You decide to extract two features: first, the length of the string, and, second, a numerical encoding of the first letter of the string, obtained using the function LabelEncoder()

In [ ]:
# Create a feature that contains the length of the string
proteins['len'] = proteins['seq'].apply(lambda s: len(s))

# Create a feature encoding the first letter of the string
proteins['first'] =  LabelEncoder().fit_transform(
  proteins['seq'].apply(lambda s: list(s)[0]))

# Extract scores from the fitted LoF object, compute its AUC
scores_lof = lof_detector.negative_outlier_factor_
print(auc(proteins['label']=='IMMUNE SYSTEM', scores_lof))

# Fit a 1-class SVM, extract its scores, and compute its AUC
svm = OneClassSVM().fit(proteins[['len', 'first']])
scores_svm = svm.score_samples(proteins[['len', 'first']])
print(auc(proteins['label']=='IMMUNE SYSTEM', scores_svm))